In [ ]:
def stitch_img(imgs):
    '''Stitch a list of images together.
    
    Input: 
        imgs: a list of images.
    Output:
        stitched_img: a single stiched image.
        
    TODO: implement the stitch_img function. 
    '''
    left_img = imgs[1]
    center_img = imgs[0]
    right_img = imgs[2]
    
    
    #img0 = cv2.imread('mountain_left.png')
    #img1 = cv2.imread('mountain_center.png')
    img1 = left_img
    img0 = center_img

    pts1, pts2, matches, kp1, kp2 = genSIFTMatchPairs(img0, img1)
    
    inliers_idx, H = RANSAC(pts1, pts2, 500, 20)

    new_matches = []
    for i in range(len(inliers_idx)):
        new_matches.append(matches[inliers_idx[i]])
    
    matching_result = cv2.drawMatches(img0, kp1, img1, kp2, new_matches, None, flags=2, matchColor=(0,0,255))
    #plt.imshow(cv2.cvtColor(matching_result, cv2.COLOR_BGR2RGB))
    
    result = cv2.warpPerspective(img0, H, (img0.shape[1] + img1.shape[1], img0.shape[0])) 
    result[0:img1.shape[0], 0:img1.shape[1]] = img1
    #plt.imshow(cv2.cvtColor(result.astype("uint8"), cv2.COLOR_BGR2RGB));
    
    
   
    
    # add another picture
    #img3 = cv2.imread('mountain_center.png')
    #img2 = cv2.imread('mountain_right.png')
    
    img2 = center_img
    img3 = right_img

    pts1, pts2, matches, kp1, kp2 = genSIFTMatchPairs(img3, img2)
    
    inliers_idx, H = RANSAC(pts1, pts2, 500, 20)

    new_matches = []
    for i in range(len(inliers_idx)):
        new_matches.append(matches[inliers_idx[i]])

    #matching_result = cv2.drawMatches(img3, kp1, img2, kp2, new_matches, None, flags=2, matchColor=(0,0,255))
    #plt.imshow(cv2.cvtColor(matching_result, cv2.COLOR_BGR2RGB))
    
    result2 = cv2.warpPerspective(img3, H, (img3.shape[1] + img2.shape[1], img3.shape[0])) 
    result2[0:img2.shape[0], 0:img2.shape[1]] = img2
    #plt.imshow(cv2.cvtColor(result2.astype("uint8"), cv2.COLOR_BGR2RGB));
    
    
    #adding together
    #img2 = result
    #img3 = result2
    
    '''
    final = result
    final2 = result2
    pts1, pts2, matches, kp1, kp2 = genSIFTMatchPairs(final2, result)
    
    inliers_idx, H = RANSAC(pts1, pts2, 500, 20)

    new_matches = []
    for i in range(len(inliers_idx)):
        new_matches.append(matches[inliers_idx[i]])
    
    matching_result = cv2.drawMatches(final2, kp1, final, kp2, new_matches, None, flags=2, matchColor=(0,0,255))
    plt.imshow(cv2.cvtColor(matching_result, cv2.COLOR_BGR2RGB))
    
    result3 = cv2.warpPerspective(final2, H, (final2.shape[1] + final.shape[1], final.shape[0])) 
    #result3[0:result.shape[0], 0:result.shape[1]] = result
    plt.imshow(cv2.cvtColor(result3.astype("uint8"), cv2.COLOR_BGR2RGB));
    '''
    
    # method 2 add the method together
    
    mask_left_center = binary_mask(result)
    i = 0
    for i in range(result.shape[1]):
        if mask_left_center[0][i] == 0:
            break
    
    final = np.zeros([result.shape[0], result.shape[1] + right_img.shape[1] ,3])
    final_right = np.zeros([right_img.shape[0], right_img.shape[1],3])
    final_right[:,:,:] = result2[:, center_img.shape[1]:,:]
    
    final[:, 0:result.shape[1],:] = result[:,:,:]
    #final[:,average:average+final_right.shape[1],:] = final_right[:,0:,:]
    final[:,i:i+final_right.shape[1],:] = final_right[:,0:,:]
    return result, result2 , final